In [2]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

In [3]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_02.csv")

In [5]:
df_sample = df[df["region"]=="CENTRE VAL DE LOIRE"]

In [36]:
df_stations_old = df_sample.groupby(["NUM_POSTE"]).max()
print(len(df_stations_old))
display(df_stations_old)

39


,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,date
NUM_POSTE,,,,,,,,,,,
4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,2022,12,6.8,41,Loir-et-Cher,CENTRE VAL DE LOIRE,2022-12-31
4136001,LA MURE-ARGENS,43.977000,6.520167,920,2022,12,8.3,41,Loir-et-Cher,CENTRE VAL DE LOIRE,2022-12-31
4157001,PUIMOISSON,43.871167,6.168833,775,1993,12,17.3,41,Loir-et-Cher,CENTRE VAL DE LOIRE,1993-05-31
4173002,ST ANDRE ALPES,43.958333,6.511667,882,2005,12,10.8,41,Loir-et-Cher,CENTRE VAL DE LOIRE,2005-06-30
4192001,ST MICHEL L OBS,43.933333,5.713333,650,2003,12,14.4,41,Loir-et-Cher,CENTRE VAL DE LOIRE,2003-10-31
18015003,AUBIGNY-SUR-NERE,47.499500,2.427333,176,2022,12,18.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18033001,BOURGES,47.059167,2.359833,161,2022,12,23.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18092001,AVORD,47.052167,2.642167,175,2022,12,23.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18125004,LERE,47.457833,2.893167,139,2022,12,14.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31


In [37]:
# filter out station qui n'ont pas de données récentes (2022)
mask = df_stations_old["Year"]!=df_sample["Year"].max()
df_stations_old = df_stations_old[mask]
stations_old = df_stations_old.index.to_list()
df_recent = df_sample[~df_sample["NUM_POSTE"].isin(stations_old)]

In [41]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))
df_recent.groupby(["NUM_POSTE"]).max()

30


,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,date
NUM_POSTE,,,,,,,,,,,
4134002,LA MOTTE DU CAIRE,44.326667,6.030500,665,2022,12,6.8,41,Loir-et-Cher,CENTRE VAL DE LOIRE,2022-12-31
4136001,LA MURE-ARGENS,43.977000,6.520167,920,2022,12,8.3,41,Loir-et-Cher,CENTRE VAL DE LOIRE,2022-12-31
18015003,AUBIGNY-SUR-NERE,47.499500,2.427333,176,2022,12,18.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18033001,BOURGES,47.059167,2.359833,161,2022,12,23.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18092001,AVORD,47.052167,2.642167,175,2022,12,23.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18125004,LERE,47.457833,2.893167,139,2022,12,14.0,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18172003,ORVAL RAD,46.731000,2.467333,165,2022,12,13.7,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18175003,OUROUER,46.925333,2.803333,221,2022,12,18.7,18,Cher,CENTRE VAL DE LOIRE,2022-12-31
18187004,PREVERANGES,46.425333,2.236500,462,2022,12,18.7,18,Cher,CENTRE VAL DE LOIRE,2022-12-31


In [54]:
# check continuité des données sur 7 ans
stations = df_recent["NUM_POSTE"].unique()
to_drop = []
for num_station in stations:
    df_station = df_recent[df_recent["NUM_POSTE"]==num_station]

    for year in range(2016,2022+1,1):
        nb_missing_month = 0
        try:
            df_station_year = df_station[df_station["Year"]==year]
            nb_missing_month += 12-df_station_year["Month"].count()
            if nb_missing_month > 0:
                to_drop.append(num_station)
                print(f"Drop station N°{num_station}")
                print(f"{nb_missing_month} manquants en {year}\n\n")
            else:
                print(f"Station N°{num_station} has {df_station_year["Month"].count()} months in {year}")

        except Exception as e:
            to_drop.append(num_station)
            print(e)
            print()
            print(f"Drop station N°{num_station}")
            print(f"{year} est manquante\n\n")

    print()

Station N°4134002 has 12 months in 2016
Station N°4134002 has 12 months in 2017
Station N°4134002 has 12 months in 2018
Station N°4134002 has 12 months in 2019
Station N°4134002 has 12 months in 2020
Station N°4134002 has 12 months in 2021
Station N°4134002 has 12 months in 2022

Station N°4136001 has 12 months in 2016
Station N°4136001 has 12 months in 2017
Station N°4136001 has 12 months in 2018
Station N°4136001 has 12 months in 2019
Station N°4136001 has 12 months in 2020
Station N°4136001 has 12 months in 2021
Station N°4136001 has 12 months in 2022

Station N°18015003 has 12 months in 2016
Station N°18015003 has 12 months in 2017
Station N°18015003 has 12 months in 2018
Station N°18015003 has 12 months in 2019
Station N°18015003 has 12 months in 2020
Station N°18015003 has 12 months in 2021
Station N°18015003 has 12 months in 2022

Station N°18033001 has 12 months in 2016
Station N°18033001 has 12 months in 2017
Station N°18033001 has 12 months in 2018
Station N°18033001 has 12 m

In [55]:
to_drop

[41173003]

In [6]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12645 entries, 11437 to 181730
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   NUM_POSTE                      12645 non-null  int64  
 1   NOM_USUEL                      12645 non-null  object 
 2   LAT                            12645 non-null  float64
 3   LON                            12645 non-null  float64
 4   ALTI                           12645 non-null  int64  
 5   AAAAMM                         12645 non-null  object 
 6   Year                           12645 non-null  int64  
 7   Month                          12645 non-null  int64  
 8   precip_cumul_mensu             12634 non-null  float64
 9   temp_mean_mensu                12628 non-null  float64
 10  temp_max_mensu                 12629 non-null  float64
 11  temp_min_mensu                 12629 non-null  float64
 12  NBJGELEE                       12634 non-null 

In [7]:
df_sample.groupby(["NUM_POSTE"]).count().sort_values(by="NOM_USUEL")

,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,temp_max_mensu,...,vent_speed_inst_moy_mensu,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region
NUM_POSTE,,,,,,,,,,,,,,,,,,,,,
36089003,4,4,4,4,4,4,4,4,4,4,...,4,0,0,0,0,0,0,4,4,4
28305001,11,11,11,11,11,11,11,11,11,11,...,11,11,0,11,0,0,0,11,11,11
37122003,33,33,33,33,33,33,33,33,33,33,...,33,29,3,29,29,3,29,33,33,33
41249001,54,54,54,54,54,54,54,53,53,53,...,54,54,0,54,54,0,54,54,54,54
4157001,59,59,59,59,59,59,59,53,51,51,...,59,58,58,58,28,28,28,59,59,59
41173003,75,75,75,75,75,75,75,75,75,75,...,75,75,75,75,75,75,75,75,75,75
37261009,131,131,131,131,131,131,131,131,131,131,...,131,131,10,131,0,0,0,131,131,131
28280002,134,134,134,134,134,134,134,134,134,134,...,134,133,0,133,93,0,93,134,134,134
4173002,176,176,176,176,176,176,176,175,176,176,...,176,176,176,176,176,176,176,176,176,176


In [8]:
# PREPROCESSING DATE

df_sample["AAAAMM"] = pd.to_datetime(df_sample["AAAAMM"])

def last_day_of_month(aaaamm):
    """
    prend une date et retourne une string avec le jour en dernier jour du mois
    """
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_sample["date"] = df_sample["AAAAMM"].apply(last_day_of_month)
df_sample["date"] = pd.to_datetime(df_sample["date"])


C:\Users\alexm\AppData\Local\Temp\ipykernel_21636\1758918003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["AAAAMM"] = pd.to_datetime(df_sample["AAAAMM"])
C:\Users\alexm\AppData\Local\Temp\ipykernel_21636\1758918003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["date"] = df_sample["AAAAMM"].apply(last_day_of_month)
C:\Users\alexm\AppData\Local\Temp\ipykernel_21636\1758918003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [9]:
to_keep = ["NUM_POSTE","NOM_USUEL","LAT","LON","ALTI","Year","Month","vent_speed_inst_moy_mensu","departement_num","departement_name","region","date"]
to_drop = [col for col in df_sample if col not in to_keep]

In [10]:
to_drop

['AAAAMM',
 'precip_cumul_mensu',
 'temp_mean_mensu',
 'temp_max_mensu',
 'temp_min_mensu',
 'NBJGELEE',
 'humi_min_mensu',
 'humi_max_mensu',
 'humi_moy_mensu',
 'vent_speed_inst_max_mensu',
 'vent_dir_inst',
 'vent_nbjour_inst_speedsup10ms',
 'vent_speed_10mn_max_mensu',
 'vent_dir_10mn',
 'vent_nbjour_10mn_speedsup10ms']

In [11]:
df_sample = df_sample.drop(to_drop,axis=1)

In [12]:
len(df_sample["NUM_POSTE"].unique())

39

In [13]:
df_sample.isna().sum()

NUM_POSTE                    0
NOM_USUEL                    0
LAT                          0
LON                          0
ALTI                         0
Year                         0
Month                        0
vent_speed_inst_moy_mensu    0
departement_num              0
departement_name             0
region                       0
date                         0
dtype: int64

In [15]:
def get_wind_forecast(time, data):

    """
    INPUT =  nombre de mois (time) à prédire et le set de données

    --> prédit la force du vent à horizon "time"
    
    OUTPUT =  le df de test, les prédictions, la MAE et la MAPE
    """
    # calcule date de split 
    date_limite = data.index.max() - pd.DateOffset(months=time)

    # split train / test
    train = data[data.index <= date_limite]
    test = data[data.index > date_limite]

    # split variables exogènes / target
    to_keep = ["LON","LAT","ALTI"]

    var_exog_train = train[to_keep]
    y_train = train["vent_speed_inst_moy_mensu"]

    var_exog_test = test[to_keep]
    y_test = test["vent_speed_inst_moy_mensu"]

    # Vérifier la présence de NaN
    print(var_exog_train.isna().sum())

    # Vérifier la présence de valeurs infinies
    print(np.isinf(var_exog_train).sum())


    # training
    model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_model = model.fit(maxiter=1000)

    # prédictions
    forecast = sarima_model.get_forecast(steps = time, exog = var_exog_test)
    predicted_values = forecast.predicted_mean

    # évaluation
    mae = mean_absolute_error(y_test, predicted_values)
    mape = mean_absolute_percentage_error(y_test, predicted_values)

    # rendu
    return test, predicted_values, mae, mape


In [ ]:
# créé un dictionnaire contenant un dataframe par numéro de station, puis le dataframe d'éval du modèle
# chaque dataset est réindexé sur la date avec fréquence de saisonnalité mensuelle

stations = df_sample["NUM_POSTE"].unique()

dict_df = {}

nb_month = 84

for station in stations:

    df_station = df_sample[df_sample["NUM_POSTE"]==station]
    df_station.set_index("date", inplace=True)
    df_station = df_station.asfreq("ME")  # 'ME' pour mensuel

    # on garde que les stations qui ont minimum 7 ans de données
    if len(df_station) >= nb_month:

        df_station_eval, y_pred, mae, mape = get_wind_forecast(nb_month, df_station)
        df_station_eval["pred_7_ans"] = y_pred
        df_station_eval["mae"] = mae
        df_station_eval["mape"] = mape
        dict_df[station] = (df_station, df_station_eval)


    


In [ ]:
'''for station in stations:


    df_station = df_sample[df_sample["NUM_POSTE"]==station]
    df_station.set_index("date", inplace=True)
    df_station = df_station.asfreq("ME")  # 'ME' pour mensuel
    
    if len(df_station)-1 >= 84:
        print(f"Station: {station}")
        print(f"nb_mois = {len(df_station)-1}")
        print()
        count_84 += 1
        if df_station.isna().any().any() == True:
            count_NA += 1

print(f"total station avec mois > 84: {count_84}")
print(f"total station > 84 avec NA: {count_NA}")'''

In [18]:
stations = df_sample["NUM_POSTE"].unique()

df_station = df_sample[df_sample["NUM_POSTE"]==stations[0]]

df_station.groupby(["Year"]).count()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,date
Year,,,,,,,,,,,
1990,12,12,12,12,12,12,12,12,12,12,12
1991,12,12,12,12,12,12,12,12,12,12,12
1992,11,11,11,11,11,11,11,11,11,11,11
1993,11,11,11,11,11,11,11,11,11,11,11
1994,12,12,12,12,12,12,12,12,12,12,12
1995,12,12,12,12,12,12,12,12,12,12,12
1996,12,12,12,12,12,12,12,12,12,12,12
1997,12,12,12,12,12,12,12,12,12,12,12
1998,6,6,6,6,6,6,6,6,6,6,6


In [23]:
df_year = df_station[df_station["Year"]==2022]
df_year = df_year["Year"]
df_year.count()


12